## Análise de dados de vendas para segmentação de clientes, recomendação de produtos e previsão de demanda.

Dados disponíveis em: https://www.kaggle.com/datasets/rohitsahoo/sales-forecasting

### 2) Sistema de recomendação

Importando as bibliotecas:

In [1]:
from time import time, strftime, gmtime
global_start = time()

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from re import sub

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

Importando os dados:

In [2]:
df = pd.read_csv('../data/train.csv')

df.sample(1)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
1651,1652,CA-2015-135699,29/08/2015,29/08/2015,Same Day,HH-15010,Hilary Holden,Corporate,United States,San Francisco,California,94110.0,West,OFF-PA-10003001,Office Supplies,Paper,Xerox 1986,13.36


A análise de cesta de mercado consiste em avaliar um item comprado e, baseado nas compras de outros usuários, indicar quais itens podem também agradar.<br>
É uma análise simples, porém efetiva quando não existem dados sobre o histórico de compras do usuário ou sobre as conexões entre os produtos.<br>
O primeiro passo é transformar os dados para avaliar quais produtos foram comprados em cada ordem de compra:

In [3]:
# Verificando a relação 'Order ID' e 'Customer ID':
order_df = df[['Order ID', 'Customer ID']].sort_values(['Order ID', 'Customer ID']).drop_duplicates()
# Cada cliente pode ter várias ordens, mas uma ordem só pode ter um cliente, ou seja, 'Order ID' deve ser único.
order_df[order_df['Order ID'].duplicated()]

,Order ID,Customer ID


In [4]:
df_un = df.copy()

In [5]:
#Verificando o perfil dos dados:
describe = pd.DataFrame(columns=['NaN', 'Null', 'Unique'])
for n in df_un.columns:
    describe = pd.concat([describe, pd.DataFrame({'NaN': df_un[n].isna().sum(), 
                                                  'Null': df_un[n].isnull().sum(), 
                                                  'Unique': len(df_un[n].unique())}, index=[n])])
describe

,NaN,Null,Unique
Row ID,0,0,9800
Order ID,0,0,4922
Order Date,0,0,1230
Ship Date,0,0,1326
Ship Mode,0,0,4
Customer ID,0,0,793
Customer Name,0,0,793
Segment,0,0,3
Country,0,0,1
City,0,0,529


O núm. de ID difere do núm de nomes, ou seja, existem produtos com dois IDs.

In [6]:
print('Nomes:',len(df_un['Product Name'].unique()), 'IDs:', len(df_un['Product ID'].unique()))

df_un[['Product Name', 'Product ID']].drop_duplicates().sort_values('Product Name')[df_un['Product Name'].duplicated() == True].head()

Nomes: 1849 IDs: 1861


,Product Name,Product ID
1989,"#10- 4 1/8"" x 9 1/2"" Recycled Envelopes",OFF-EN-10000781
2531,Avery Non-Stick Binders,OFF-BI-10000829
283,Easy-staple paper,OFF-PA-10000474
1225,Easy-staple paper,OFF-PA-10001685
1747,Easy-staple paper,OFF-PA-10004947


In [7]:
# Alterando o ID para o primeiro ID encontrado para o nome em questão:
df_un['Product ID'] = df_un['Product Name'].apply(lambda x: df_un[df_un['Product Name'] == x]['Product ID'].iloc[0])

print('Nomes:',len(df_un['Product Name'].unique()), 'IDs:', len(df_un['Product ID'].unique()))

df_un[['Product Name', 'Product ID']].drop_duplicates().sort_values('Product Name')[df_un['Product Name'].duplicated() == False]

Nomes: 1849 IDs: 1817


,Product Name,Product ID
1708,"""While you Were Out"" Message Book, One Form pe...",OFF-PA-10003424
355,"#10 Gummed Flap White Envelopes, 100/Box",OFF-EN-10001137
3042,#10 Self-Seal White Envelopes,OFF-EN-10002312
2499,"#10 White Business Envelopes,4 1/8 x 9 1/2",OFF-EN-10004483
985,"#10- 4 1/8"" x 9 1/2"" Recycled Envelopes",OFF-EN-10000461
...,...,...
1502,iKross Bluetooth Portable Keyboard + Cell Phon...,TEC-PH-10001300
5307,iOttie HLCRIO102 Car Mount,TEC-PH-10002583
6235,iOttie XL Car Mount,TEC-PH-10000127
1110,invisibleSHIELD by ZAGG Smudge-Free Screen Pro...,TEC-PH-10003589


In [8]:
df_un['Product Name'] = df_un['Product ID'].apply(lambda x: df_un[df_un['Product ID'] == x]['Product Name'].iloc[0])

print('Nomes:',len(df_un['Product Name'].unique()), 'IDs:', len(df_un['Product ID'].unique()))

Nomes: 1817 IDs: 1817


O número de produtos é relativamente grande em comparação ao número de registros.<br>
Como essa proporção pode atrapalhar os resultados, os produtos serão melhor avaliados para ver se há possibilidade de diminuir esse número total de produtos.

In [9]:
# Criando o dataframe com códigos únicos:
df_item = pd.DataFrame(columns=['prod_id', 'prod_name', 'category', 'sub_category', 'number'])
df_item['prod_name'] = df_un[['Product Name', 'Product ID']].drop_duplicates().sort_values('Product Name')[df_un['Product Name'].duplicated() == False]['Product Name']
df_item['prod_id'] = df_un[['Product Name', 'Product ID']].drop_duplicates().sort_values('Product Name')[df_un['Product Name'].duplicated() == False]['Product ID']
df_item.reset_index(inplace=True, drop=True)

# Dividindo o código do produto:
for n in range(len(df_item)):
    df_item['category'][n] = df_item['prod_id'][n].split('-')[0]
    df_item['sub_category'][n] = df_item['prod_id'][n].split('-')[1]
    df_item['number'][n] = df_item['prod_id'][n].split('-')[2]

In [10]:
df_item.sort_values('prod_name').head()

,prod_id,prod_name,category,sub_category,number
0,OFF-PA-10003424,"""While you Were Out"" Message Book, One Form pe...",OFF,PA,10003424
1,OFF-EN-10001137,"#10 Gummed Flap White Envelopes, 100/Box",OFF,EN,10001137
2,OFF-EN-10002312,#10 Self-Seal White Envelopes,OFF,EN,10002312
3,OFF-EN-10004483,"#10 White Business Envelopes,4 1/8 x 9 1/2",OFF,EN,10004483
4,OFF-EN-10000461,"#10- 4 1/8"" x 9 1/2"" Recycled Envelopes",OFF,EN,10000461


In [11]:
# Vendas por tipo de cliente:
df_un.Segment.value_counts()

Consumer       5101
Corporate      2953
Home Office    1746
Name: Segment, dtype: int64

In [12]:
# Salvando o novo dataframe para utilizar na aplicação:
df_un[['Order ID', 'Order Date', 'Segment', 'State', 'City', 
       'Product Name', 'Category', 'Sub-Category', 'Sales']].to_csv('../data/vendas.csv')

#### Aplicando TransactionEncoder () e o algoritmo apriori para quantificar as associações:

In [13]:
# Gerando uma lista de listas para utilizar o encoder:
dataset = []
uniques = df_un['Customer ID'].unique()
for n in uniques:
    dataset.append(df_un[df_un['Customer ID'] == n]['Product Name'].to_list())

In [14]:
enc = TransactionEncoder()

enc_dataset = enc.fit(dataset).transform(dataset, sparse=True)

mba_df = pd.DataFrame.sparse.from_spmatrix(enc_dataset, columns=enc.columns_)

In [15]:
supp_df = apriori(mba_df, min_support=0.0015, use_colnames=True)
supp_df['Len'] = supp_df.itemsets.apply(lambda x: len(x))

supp_df.sort_values(['Len', 'support'], ascending=False).head(3)

print('shape:', supp_df.shape)
supp_df.sort_values('support', ascending=False).head(3)

shape: (3677, 3)


,support,itemsets,Len
1397,0.055485,(Staple envelope),1
504,0.055485,(Easy-staple paper),1
1403,0.054224,(Staples),1


O item 'Staple envelope' aparece nas compras de aproximadamente 60% dos clientes.

Essa avaliação também poderia ter sido feita por transação/OrderID e avaliar quantas vezes um item aparece.

In [16]:
df_rules = association_rules(supp_df, metric='lift', min_threshold=1).sort_values(
    'lift', ascending=False).reset_index(drop=True)

df_rules['ant_len'] = df_rules.antecedents.apply(lambda x: len(x))
df_rules['con_len'] = df_rules.consequents.apply(lambda x: len(x))

df_rules = df_rules[df_rules['ant_len'] == 1][df_rules['con_len'] == 1].reset_index(drop=True)

In [17]:
produto = frozenset({df_un['Product Name'].sample(1).to_list()[0]})

df_rules.sort_values(['antecedents', 'conviction'], ascending=False)[df_rules.antecedents == produto].head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,ant_len,con_len
1098,(Eldon Shelf Savers Cubes and Bins),"(Safco Value Mate Series Steel Bookcases, Bake...",0.010088,0.007566,0.002522,0.25,33.041667,0.002446,1.323245,1,1
1515,(Eldon Shelf Savers Cubes and Bins),(Poly Designer Cover & Back),0.010088,0.008827,0.002522,0.25,28.321429,0.002433,1.321564,1,1
2385,(Eldon Shelf Savers Cubes and Bins),"(Acco Pressboard Covers with Storage Hooks, 14...",0.010088,0.013871,0.002522,0.25,18.022727,0.002382,1.314838,1,1
2541,(Eldon Shelf Savers Cubes and Bins),(Hon Deluxe Fabric Upholstered Stacking Chairs...,0.010088,0.015132,0.002522,0.25,16.520833,0.002369,1.313157,1,1


Apesar de muitos produtos serem similares, cada nome é único e o retorno para esse volume de dados fica prejudicado por conta do excesso de produtos diferentes.

Algumas opções são:<br>
+ Dividir os dataframes por segmento de cliente;
+ Fazer as associações por sub-categoria e recomendar os 5 mais vendidos na mesma categoria e na categoria consequente;
+ Fazer as associações por sub-categoria e recomendar 5 produtos aleatórios na mesma categoria e na categoria consequente.

#### Para a aplicação a ser desenvolvida a associação será feita com base no tipo de cliente e no produto. Após essa associação, o retorno será os 5 produtos relacionados e os 5 mais vendidos na mesma categoria.

Calculando o support e fazendo as associações para gerar os dataframes por tipo de cliente:

In [18]:
# Segmentos e min_support:
supports = {'Home Office': 0.0075, 'Corporate': 0.0045, 'Consumer': 0.0025}

# Em caso de falha no kernel, basta aumentar os valores de min_support, pois o método gera um array maior
# quanto menor o valor.

for n in supports.keys():
    df_cat = df_un[df_un.Segment == n]
    
    # Gerando uma lista de listas para utilizar o encoder:
    dataset = []
    uniques = df_cat['Customer ID'].unique() 
    for m in uniques:
        dataset.append(df_cat[df_cat['Customer ID'] == m]['Product Name'].to_list())

    # Aplicando a transformação dos dados:
    enc = TransactionEncoder()
    enc_dataset = enc.fit(dataset).transform(dataset, sparse=True)
    mba_df = pd.DataFrame.sparse.from_spmatrix(enc_dataset, columns=enc.columns_)

    # Calculando os suportes:
    supp_df = apriori(mba_df, min_support=supports[n], use_colnames=True)
    supp_df['Len'] = supp_df.itemsets.apply(lambda x: len(x))
    
    # Buscando as associações:
    df_rules = association_rules(supp_df, metric='lift', min_threshold=1).sort_values(
        'lift', ascending=False).reset_index(drop=True)
    df_rules['ant_len'] = df_rules.antecedents.apply(lambda x: len(x))
    df_rules['con_len'] = df_rules.consequents.apply(lambda x: len(x))
    df_rules = df_rules[df_rules['ant_len'] == 1][df_rules['con_len'] == 1].reset_index(drop=True)
    
    # Alterando o dataframe para salvar apenas os dados necessários:
    associacoes = df_rules[['antecedents', 'consequents', 'conviction']].sort_values(
        ['antecedents', 'conviction'], ascending=False)

    associacoes['antecedents'] = associacoes['antecedents'].apply(lambda x: list(x)[0])
    associacoes['consequents'] = associacoes['consequents'].apply(lambda x: list(x)[0])
    
    # Salvando o dataframe de associacoes por tipo de cliente:
    file_name = n.lower().replace(' ', '_') + '_associacoes'
    associacoes.to_csv(f'./../data/{file_name}.csv')

Gerando os dataframes com os itens mais vendidos por sub-categoria, porém apenas os itens que possuem pelo menos uma associação consequente:

In [19]:
for n in df_un.Segment.unique():
    df_cat = df_un[df_un.Segment == n]
    n_orders = len(df_un['Order ID'].unique())
    df_sales = pd.DataFrame(columns=['Product Name', 'Sub-Category', 'Frequency'])
    df_sales['Product Name'] = df_cat['Product Name'].unique()

    df_sales['Frequency'] = df_sales['Product Name'].apply(
        lambda x: df_cat['Product Name'].value_counts()[x]/n_orders)
    df_sales['Sub-Category'] = df_sales['Product Name'].apply(
        lambda x: df_cat[df_cat['Product Name'] == x]['Sub-Category'].iloc[0])
    
    assocs_name = n.lower().replace(' ', '_') + '_associacoes'
    assocs = pd.read_csv(f'./../data/{assocs_name}.csv', index_col=0)
    
    df_sales = df_sales[df_sales['Product Name'].isin(assocs.antecedents.unique())].reset_index(drop=True)
    
    file_name = n.lower().replace(' ', '_') + '_prop_vendas'
    df_sales.sort_values(['Sub-Category', 'Frequency'], ascending=False).to_csv(f'./../data/{file_name}.csv')

#### Criando função para fazer a análise a partir do produto selecionado:

Para a função utilizada na aplicação será feita a recomendação de até 5 consequentes com maior valor de 'conviction' e os 5 mais vendidos para a sub-categoria do produto escolhido.

In [20]:
def indicados(segmento, produto, associacoes, prop_vendas):
    categoria = prop_vendas[prop_vendas['Product Name'] == produto]['Sub-Category'].iloc[0]
    
    vendidos = prop_vendas[prop_vendas['Sub-Category'] == categoria][
        prop_vendas['Product Name'] != produto]['Product Name'].to_list()

    recomendados = associacoes[associacoes.antecedents == produto]['consequents'].to_list()

    diff = len(vendidos) - len(recomendados)
    if diff > 0:
        [recomendados.append('Sem mais recomendações') for n in range(diff)]
    elif diff < 0:
        [vendidos.append('Sem mais recomendações') for n in range(-diff)]
    else:
        pass

    df = pd.DataFrame({'Quem comprou esse produto também comprou': recomendados, 
                   'Você também pode gostar de': vendidos})
    
    return df.head(5)

In [21]:
# Testando o processo:
seg = np.random.choice(['Home Office', 'Corporate', 'Consumer'])
associacao_csv = seg.lower().replace(' ', '_') + '_associacoes'
prop_vendas_csv = seg.lower().replace(' ', '_') + '_prop_vendas'

associacoes = pd.read_csv(f'./../data/{associacao_csv}.csv', index_col=0)
prop_vendas = pd.read_csv(f'./../data/{prop_vendas_csv}.csv', index_col=0)

produto = np.random.choice(prop_vendas['Product Name'].unique())

indicados(seg, produto, associacoes, prop_vendas)

,Quem comprou esse produto também comprou,Você também pode gostar de
0,"GBC Twin Loop Wire Binding Elements, 9/16"" Spi...",DMI Eclipse Executive Suite Bookcases
1,Sem mais recomendações,"Sauder Camden County Collection Libraries, Pla..."
2,Sem mais recomendações,O'Sullivan Living Dimensions 2-Shelf Bookcases
3,Sem mais recomendações,O'Sullivan Living Dimensions 5-Shelf Bookcases
4,Sem mais recomendações,"Safco Value Mate Series Steel Bookcases, Baked..."


In [22]:
f'Tempo de execução: {strftime("%H:%M:%S", gmtime(time()-global_start))}'

'Tempo de execução: 00:00:38'

### Fim.